<a href="https://colab.research.google.com/github/spozi/gpu-svgpm/blob/main/GPFeatureGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Thu Aug  5 12:24:42 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# This get the RAPIDS-Colab install files and test check your GPU.  Run this and the next cell only.
# Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/env-check.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 277, done.
remote: Counting objects: 100% (106/106), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 277 (delta 59), reused 40 (delta 20), pack-reused 171
Receiving objects: 100% (277/277), 80.66 KiB | 1.37 MiB/s, done.
Resolving deltas: 100% (121/121), done.
***********************************************************************
Woo! Your instance has the right kind of GPU, a Tesla T4!
***********************************************************************



In [ ]:
# This will update the Colab environment and restart the kernel.  Don't run the next cell until you see the session crash.
!bash rapidsai-csp-utils/colab/update_gcc.sh
import os
os._exit(00)

Updating your Colab environment.  This will restart your kernel.  Don't Panic!
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [62.9 kB]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages
Get:11 https://de

In [ ]:
# This will install CondaColab.  This will restart your kernel one last time.  Run this cell by itself and only run the next cell once you see the session crash.
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:36
🔁 Restarting kernel...


In [ ]:
# you can now run the rest of the cells as normal
import condacolab
condacolab.check()

✨🍰✨ Everything looks OK!


In [ ]:
# Installing RAPIDS is now 'python rapidsai-csp-utils/colab/install_rapids.py <release> <packages>'
# The <release> options are 'stable' and 'nightly'.  Leaving it blank or adding any other words will default to stable.
# The <packages> option are default blank or 'core'.  By default, we install RAPIDSAI and BlazingSQL.  The 'core' option will install only RAPIDSAI and not include BlazingSQL, 
!python rapidsai-csp-utils/colab/install_rapids.py stable
import os
os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'
os.environ['CONDA_PREFIX'] = '/usr/local'

Installing RAPIDS Stable 21.06
Starting the RAPIDS+BlazingSQL install on Colab.  This will take about 15 minutes.
Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - cudatoolkit=11.0
    - gcsfs
    - llvmlite
    - openssl
    - python=3.7
    - rapids-blazing=21.06


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    abseil-cpp-20210324.1      |       h9c3ff4c_0        1015 KB  conda-forge
    aiohttp-3.7.4.post0        |   py37h5e8e339_0         625 KB  conda-forge
    anyio-3.3.0                |   py37h89c1867_0         146 KB  conda-forge
    argon2-cffi-20.1.0         |   py37h5e8e339_2      

In [ ]:
!pip install -U deap

     |████████████████████████████████| 160 kB 8.3 MB/s 


Generate huge data

In [ ]:
from sklearn.datasets import make_blobs
X, y = make_blobs(n_samples=100000, centers=5, n_features=1000, random_state=0)
X_32 = np.asarray(X, dtype='f')

Store data in GPU memory

In [ ]:
import numpy as np
import cupy as cp

x_gpu = cp.asarray(X_32)
squared_diff = cp.ElementwiseKernel(
   'float32 x, float32 y',
   'float32 z',
   'z = (x - y) * (x - y)',
   'squared_diff')

squared_diff(x_gpu, x_gpu)


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
resultant_gp_matrix = cp.ElementwiseKernel(
    'float32 x',
    'float32 z'
) 

GP Generator is here, in progress.

In [ ]:
import random
import operator
import math
import statistics

import numpy as np

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp


# Define new functions
def protectedDiv(left, right):
    try:
        return left / right
    except ZeroDivisionError:
        return 1

adfset2 = gp.PrimitiveSet("ADF2", 2)
adfset2.addPrimitive(operator.add, 2)
adfset2.addPrimitive(operator.sub, 2)
adfset2.addPrimitive(operator.mul, 2)
adfset2.addPrimitive(protectedDiv, 2)
adfset2.addPrimitive(operator.neg, 1)
adfset2.addPrimitive(math.cos, 1)
adfset2.addPrimitive(math.sin, 1)

adfset1 = gp.PrimitiveSet("ADF1", 2)
adfset1.addPrimitive(operator.add, 2)
adfset1.addPrimitive(operator.sub, 2)
adfset1.addPrimitive(operator.mul, 2)
adfset1.addPrimitive(protectedDiv, 2)
adfset1.addPrimitive(operator.neg, 1)
adfset1.addPrimitive(math.cos, 1)
adfset1.addPrimitive(math.sin, 1)
adfset1.addADF(adfset2)

adfset0 = gp.PrimitiveSet("ADF0", 2)
adfset0.addPrimitive(operator.add, 2)
adfset0.addPrimitive(operator.sub, 2)
adfset0.addPrimitive(operator.mul, 2)
adfset0.addPrimitive(protectedDiv, 2)
adfset0.addPrimitive(operator.neg, 1)
adfset0.addPrimitive(math.cos, 1)
adfset0.addPrimitive(math.sin, 1)
adfset0.addADF(adfset1)
adfset0.addADF(adfset2)

pset = gp.PrimitiveSet("MAIN", 1)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(protectedDiv, 2)
pset.addPrimitive(operator.neg, 1)
pset.addPrimitive(math.cos, 1)
pset.addPrimitive(math.sin, 1)
pset.addEphemeralConstant("rand100", lambda: random.randint(-1, 1))
pset.addADF(adfset0)
pset.addADF(adfset1)
pset.addADF(adfset2)
pset.renameArguments(ARG0='x')

psets = (pset, adfset0, adfset1, adfset2)

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Tree", gp.PrimitiveTree)

creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register('adf_expr0', gp.genFull, pset=adfset0, min_=1, max_=2)
toolbox.register('adf_expr1', gp.genFull, pset=adfset1, min_=1, max_=2)
toolbox.register('adf_expr2', gp.genFull, pset=adfset2, min_=1, max_=2)
toolbox.register('main_expr', gp.genHalfAndHalf, pset=pset, min_=1, max_=2)

toolbox.register('ADF0', tools.initIterate, creator.Tree, toolbox.adf_expr0)
toolbox.register('ADF1', tools.initIterate, creator.Tree, toolbox.adf_expr1)
toolbox.register('ADF2', tools.initIterate, creator.Tree, toolbox.adf_expr2)
toolbox.register('MAIN', tools.initIterate, creator.Tree, toolbox.main_expr)

func_cycle = [toolbox.MAIN, toolbox.ADF0, toolbox.ADF1, toolbox.ADF2]
# func_cycle = [toolbox.MAIN]

toolbox.register('individual', tools.initCycle, creator.Individual, func_cycle)
toolbox.register('population', tools.initRepeat, list, toolbox.individual)

def evalSymbReg(score):
  return score,

def evalSymbRegPop(population):
  # Evaluate each individual in population
  average_sqerrors = []
  for individual in population:
    func = toolbox.compile(expr=individual)
    points = [x/10. for x in range(-10, 10)]
    sqerrors = ((func(x) - x**4 - x**3 - x**2 - x)**2 for x in points)
    average_sqerrors.append(math.fsum(sqerrors) / len(points))
  
  #Filter individual that high than certain threshold
  ind_pop_fitness = []
  avg = statistics.mean(average_sqerrors)
  for val in average_sqerrors:
    if val > avg:
      ind_pop_fitness.append(1.0)
    else:
      ind_pop_fitness.append(val)
  return ind_pop_fitness

toolbox.register('compile', gp.compileADF, psets=psets)
# toolbox.register('compile', gp.compileADF, pset)
toolbox.register('evaluate', evalSymbReg)
toolbox.register('select', tools.selTournament, tournsize=3)
toolbox.register('mate', gp.cxOnePoint)
toolbox.register('expr', gp.genFull, min_=1, max_=2)
toolbox.register('mutate', gp.mutUniform, expr=toolbox.expr)

def main():
  random.seed(1024)
  ind = toolbox.individual()
  
  pop = toolbox.population(n=100)
  hof = tools.HallOfFame(1)
  stats = tools.Statistics(lambda ind: ind.fitness.values)
  stats.register("avg", np.mean)
  stats.register("std", np.std)
  stats.register("min", np.min)
  stats.register("max", np.max)
  
  logbook = tools.Logbook()
  logbook.header = "gen", "evals", "std", "min", "avg", "max"
  
  CXPB, MUTPB, NGEN = 0.5, 0.2, 40
  
  # # Evaluate the entire population
  #1. Compute the metric on the set of individuals
  ind_pop_fitness = evalSymbRegPop(pop)

  #2. Then, determine the best individual using toolbox
  for ind, fitness in zip(pop, ind_pop_fitness):
    ind.fitness.values = toolbox.evaluate(fitness)

  hof.update(pop)
  f = stats.compile(pop)
  record = stats.compile(pop)
  logbook.record(gen=0, evals=len(pop), **record)
  print(logbook.stream)
  
  for g in range(1, NGEN):
      # Select the offspring
      offspring = toolbox.select(pop, len(pop))
      # Clone the offspring
      offspring = [toolbox.clone(ind) for ind in offspring]
  
      # Apply crossover and mutation
      for ind1, ind2 in zip(offspring[::2], offspring[1::2]):
          for tree1, tree2 in zip(ind1, ind2):
              if random.random() < CXPB:
                  toolbox.mate(tree1, tree2)
                  del ind1.fitness.values
                  del ind2.fitness.values

      for ind in offspring:
          for tree, pset in zip(ind, psets):
              if random.random() < MUTPB:
                  toolbox.mutate(individual=tree, pset=pset)
                  del ind.fitness.values
                          
      # Evaluate the individuals with an invalid fitness
      invalids = [ind for ind in offspring if not ind.fitness.valid]

      #1. Compute the metric on the set of individuals
      ind_pop_invalid_fitness = evalSymbRegPop(invalids)

      #2. Then, determine the best individual using toolbox
      for ind, fitness in zip(invalids, ind_pop_invalid_fitness):
        ind.fitness.values = toolbox.evaluate(fitness)
              
      # Replacement of the population by the offspring
      pop = offspring
      hof.update(pop)
      record = stats.compile(pop)
      logbook.record(gen=g, evals=len(invalids), **record)
      print(logbook.stream)
  
  # print('Best individual : ', hof[0][0], hof[0].fitness)
  
  return pop, stats, hof
    
if __name__ == "__main__":
  pop, stats, hof = main()

/usr/local/lib/python3.7/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/usr/local/lib/python3.7/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'Tree' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/usr/local/lib/python3.7/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


gen	evals	std     	min     	avg     	max   
0  	100  	0.220615	0.403128	0.989312	1.6941
1  	95   	0.962781	0.328818	1.33554 	6.85956
2  	98   	0.2596  	0.328818	0.853633	1.56596
3  	97   	0.247406	0.326439	0.750048	1.10624
4  	97   	0.228297	0.328818	0.756026	1.04062
5  	96   	0.269897	0.254871	0.79825 	1.22958
6  	96   	0.252946	0.254871	0.696777	1.10624
7  	95   	0.251635	0.314221	0.674869	1      
8  	94   	0.274121	0.274004	0.684048	1.43569
9  	96   	0.286301	0.193912	0.715686	1.22958
10 	96   	0.261111	0.193912	0.687148	1.10624
11 	98   	0.246637	0.328818	0.630438	1      
12 	96   	0.39589 	0.274464	0.669163	2.12845
13 	96   	0.280005	0.165572	0.588501	1      
14 	98   	0.256704	0.165572	0.533796	1      
15 	98   	0.265338	0.165572	0.548886	1      
16 	95   	0.254774	0.165572	0.491954	1      
17 	100  	0.2923  	0.165572	0.526897	1      
18 	97   	0.28961 	0.165572	0.508439	1      
19 	98   	0.304086	0.165572	0.502169	1      
20 	97   	0.35021 	0.134078	0.501954	1      
21 	99   	0.

In [ ]:
for individual in hof:
  for i in individual:
    print(i, individual.fitness )

add(ADF2(ADF1(add(x, x), x), x), x) (0.02102276689830664,)
ADF1(sub(ARG1, ARG0), ADF1(add(ADF2(ARG0, ARG0), ADF1(neg(ARG1), ADF2(ARG1, ARG1))), protectedDiv(add(ARG1, ARG0), neg(ARG1)))) (0.02102276689830664,)
add(sub(protectedDiv(ARG0, add(ARG0, ARG1)), cos(mul(ARG0, ARG1))), mul(ARG0, ARG1)) (0.02102276689830664,)
add(mul(ARG1, ARG1), mul(sin(mul(ARG1, ARG1)), add(ARG1, protectedDiv(ARG1, ARG1)))) (0.02102276689830664,)


Version 2 GP Feature Generator

In [ ]:
import random
import operator
import math
import statistics

import numpy

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp


# Define new functions
# def protectedDiv(left, right):
#     with numpy.errstate(divide='ignore',invalid='ignore'):
#         x = numpy.divide(left, right)
#         if isinstance(x, numpy.ndarray):
#             x[numpy.isinf(x)] = 1
#             x[numpy.isnan(x)] = 1
#         elif numpy.isinf(x) or numpy.isnan(x):
#             x = 1
#     return x


# Define new functions
def protectedDiv(left, right):
    try:
        return left / right
    except ZeroDivisionError:
        return 1

pset = gp.PrimitiveSet("MAIN", 1)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(protectedDiv, 2)
pset.addPrimitive(operator.neg, 1)
pset.addPrimitive(math.cos, 1)
pset.addPrimitive(math.sin, 1)
pset.addEphemeralConstant("r12", lambda: random.randint(-1,1))
# pset.renameArguments(ARG0='x')

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Tree", gp.PrimitiveTree)
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("main_expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=5)
toolbox.register('MAIN', tools.initIterate, creator.Tree, toolbox.main_expr)

func_cycle = [toolbox.MAIN]

toolbox.register("individual", tools.initCycle, creator.Individual, func_cycle)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def evalSymbReg(score):
  return score,

def evalSymbRegPop(population):
  # Evaluate each individual in population
  average_sqerrors = []
  for individual in population:
    func = toolbox.compile(expr=individual)
    points = [x/10. for x in range(-10, 10)]
    sqerrors = ((func(x) - x**4 - x**3 - x**2 - x)**2 for x in points)
    average_sqerrors.append(math.fsum(sqerrors) / len(points))
  
  # Filter individual that high than certain threshold
  ind_pop_fitness = []
  avg = statistics.mean(average_sqerrors)
  for val in average_sqerrors:
    if val > avg:
      ind_pop_fitness.append(1.0)
    else:
      ind_pop_fitness.append(val)
  return ind_pop_fitness

psets = [pset]
toolbox.register("compile", gp.compileADF, psets=psets)
toolbox.register('evaluate', evalSymbReg)
toolbox.register('select', tools.selTournament, tournsize=3)
toolbox.register('mate', gp.cxOnePoint)
toolbox.register("expr", gp.genFull, min_=1, max_=2)
toolbox.register('mutate', gp.mutUniform, expr=toolbox.expr)

def main():
  random.seed(1024)
  ind = toolbox.individual()
  
  pop = toolbox.population(n=100)
  hof = tools.HallOfFame(1)
  stats = tools.Statistics(lambda ind: ind.fitness.values)
  stats.register("avg", numpy.mean)
  stats.register("std", numpy.std)
  stats.register("min", numpy.min)
  stats.register("max", numpy.max)

  logbook = tools.Logbook()
  logbook.header = "gen", "evals", "std", "min", "avg", "max"

  CXPB, MUTPB, NGEN = 0.5, 0.2, 40

  # # Evaluate the entire population
  #1. Compute the metric on the set of individuals
  ind_pop_fitness = evalSymbRegPop(pop)

  #2. Then, determine the best individual using toolbox
  for ind, fitness in zip(pop, ind_pop_fitness):
    ind.fitness.values = toolbox.evaluate(fitness)

  hof.update(pop)
  record = stats.compile(pop)
  logbook.record(gen=0, evals=len(pop), **record)
  print(logbook.stream)

  for g in range(1, NGEN):
    # Select the offspring
    offspring = toolbox.select(pop, len(pop))
    # Clone the offspring
    offspring = [toolbox.clone(ind) for ind in offspring]

    # Apply crossover and mutation
    for ind1, ind2 in zip(offspring[::2], offspring[1::2]):
        for tree1, tree2 in zip(ind1, ind2):
            if random.random() < CXPB:
                toolbox.mate(tree1, tree2)
                del ind1.fitness.values
                del ind2.fitness.values

    for ind in offspring:
        for tree, pset in zip(ind, psets):
            if random.random() < MUTPB:
                toolbox.mutate(individual=tree, pset=pset)
                del ind.fitness.values
                        
    # Evaluate the individuals with an invalid fitness
    invalids = [ind for ind in offspring if not ind.fitness.valid]

    #1. Compute the metric on the set of individuals
    ind_pop_invalid_fitness = evalSymbRegPop(invalids)

    #2. Then, determine the best individual using toolbox
    for ind, fitness in zip(invalids, ind_pop_invalid_fitness):
      ind.fitness.values = toolbox.evaluate(fitness)
            
    # Replacement of the population by the offspring
    pop = offspring
    hof.update(pop)
    record = stats.compile(pop)
    logbook.record(gen=g, evals=len(invalids), **record)
    print(logbook.stream)
  
  print('Best individual : ', hof[0][0], hof[0].fitness)
  return pop, stats, hof

if __name__ == "__main__":
    pop, stats, hof = main()

/usr/local/lib/python3.7/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/usr/local/lib/python3.7/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'Tree' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/usr/local/lib/python3.7/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


gen	evals	std     	min     	avg    	max   
0  	100  	0.432769	0.369013	1.16042	2.3179
1  	67   	0.345294	0.328818	0.978651	2.09449
2  	56   	1.99457 	0.165572	1.25547 	18.1314
3  	61   	0.242704	0.165572	0.690782	1.10624
4  	57   	0.266112	0.113627	0.649869	1.22958
5  	49   	0.349693	0.113627	0.629111	1.52633
6  	52   	0.281888	0.0245776	0.498762	1.10624
7  	64   	0.311743	0.0165089	0.483893	1      
8  	59   	0.329252	0.0165089	0.508528	1.10624
9  	64   	0.361524	0.0165089	0.434095	1      
10 	57   	0.340055	0.00139914	0.354443	1      
11 	58   	0.354774	0.00139914	0.34236 	1      
12 	58   	0.3459  	0.00139914	0.26646 	1      
13 	65   	0.351239	0.00139914	0.313916	1      
14 	64   	0.351375	0.00139914	0.266782	1      
15 	55   	0.379173	0.00139914	0.271482	1.07663
16 	59   	0.380116	0.00139914	0.264843	1      
17 	59   	0.378388	0.00139914	0.284306	1      
18 	71   	0.361823	0.00139914	0.298066	1      
19 	56   	0.37309 	0.00139914	0.300695	1      
20 	61   	0.389859	0.00139914	0.305

In [ ]:
for individual in hof:
  for i in individual:
    print(i, individual.fitness )

mul(add(mul(ARG0, ARG0), 1), mul(add(ARG0, sin(sub(ARG0, ARG0))), mul(1, add(1, ARG0)))) (4.237045877651919e-33,)
